# Cleaning LAQN Datasets
I will be removing the files has %100 missing values.

import and path statements below.

In [8]:
import pandas as pd
from pathlib import Path
import os
import re

# paths beloww
base_dir = Path("/Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels")

# filter logs_missin_value.csv value=100% missing value function paths below.
input_all_missing_file = base_dir / "data" / "laqn" / "missing" / "logs_missin_value.csv"
output_filtered_value_file = base_dir / "data" / "laqn" / "missing" / "value_100filtered_missing.csv"

# analyse the sites/speciest not have any value so they're not active site/species/  analyse_affected_sites function
output_notActive_site_species = base_dir / "data"/ "laqn"/ "missing"/ "notActive_site_species.csv"

# create paths for metadata removal and update the actitive site/species list below.
actv_sites_species_path = base_dir / "data" / "laqn" / "actv_sites_species.csv"
updated_actv_site_species_path = base_dir / "data" / "laqn" / "updated_actv_siteSpecies.csv"

#### 1) filter logs_missin_value.csv file according to 100 percent missing values.
- filters based on 100 value column and than creates another missing_files csv.
- filters siteCode and speciesCode columns different to find out that what site's don't have that species on their system.

In [6]:
    
def filter_missing_pollutants():
    """
    Filter the logs_missin_value.csv to create:
   value_100filtered_missing.csv - rows with 100% EmptyValuePercentage and siteCode != SpeciesCode
    
    """
    print("="*40)
    print(f"Filtering missing value logs from: {input_all_missing_file}")
    print("="*40)

    # Load the file
    df = pd.read_csv(input_all_missing_file, encoding='utf-8')
    print(f"Loaded {len(df)} rows from logs_missin_value.csv")

    # 1. Filter for 100% EmptyValuePercentage
    df_100 = df[df['EmptyValuePercentage'] == 100]
    df_100.to_csv(output_filtered_value_file, index=False)
    print(f"Saved {len(df_100)} rows with 100% missing values to: {output_filtered_value_file}")

    # 2. Filter where siteCode != SpeciesCode
    df_100 = df_100[df_100['siteCode'] != df_100['SpeciesCode']]
    df_100.to_csv(output_filtered_value_file, index=False)
    print(f"Saved {len(df_100)} rows (siteCode != SpeciesCode) to: {output_filtered_value_file}")

    print("="*100)
    print("Filtering complete.\n")

    return df_100


In [7]:

# Example usage
filter_missing_pollutants()

Filtering missing value logs from: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/laqn/missing/logs_missin_value.csv
Loaded 4136 rows from logs_missin_value.csv
Saved 3401 rows with 100% missing values to: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/laqn/missing/value_100filtered_missing.csv
Saved 3401 rows (siteCode != SpeciesCode) to: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/laqn/missing/value_100filtered_missing.csv
Filtering complete.



,filename,path,siteCode,SpeciesCode,year,month,EmptyValuePercentage
0,BL0_CO_2023-04-01_2023-04-30.csv,/Users/burdzhuchaglayan/Desktop/data science p...,BL0,CO,2023,2023_apr,100.0
2,BT4_SO2_2023-04-01_2023-04-30.csv,/Users/burdzhuchaglayan/Desktop/data science p...,BT4,SO2,2023,2023_apr,100.0
3,BT5_PM2.5_2023-04-01_2023-04-30.csv,/Users/burdzhuchaglayan/Desktop/data science p...,BT5,PM2.5,2023,2023_apr,100.0
4,BT6_PM2.5_2023-04-01_2023-04-30.csv,/Users/burdzhuchaglayan/Desktop/data science p...,BT6,PM2.5,2023,2023_apr,100.0
5,BT6_SO2_2023-04-01_2023-04-30.csv,/Users/burdzhuchaglayan/Desktop/data science p...,BT6,SO2,2023,2023_apr,100.0
...,...,...,...,...,...,...,...
4129,WM0_O3_2025-09-01_2025-09-30.csv,/Users/burdzhuchaglayan/Desktop/data science p...,WM0,O3,2025,2025_sep,100.0
4130,WM0_PM10_2025-09-01_2025-09-30.csv,/Users/burdzhuchaglayan/Desktop/data science p...,WM0,PM10,2025,2025_sep,100.0
4131,WM0_PM2.5_2025-09-01_2025-09-30.csv,/Users/burdzhuchaglayan/Desktop/data science p...,WM0,PM2.5,2025,2025_sep,100.0
4132,WM0_SO2_2025-09-01_2025-09-30.csv,/Users/burdzhuchaglayan/Desktop/data science p...,WM0,SO2,2025,2025_sep,100.0


### 2) Function for identifying site-species combinations not monitored throughout the year

- This function analyses the value_100filtered_missing.csv file to find site-species pairs that have 100% missing values for every month in a year.
- If a station has no valid values for a given species across all months (2023: 12 months, 2024: 12 months, 2025: 11 months), it is considered as not monitoring that species.
- The function groups and pivots the data to summarise which site-species pairs are consistently missing, providing a clear list of non-active monitoring combinations.
- The output includes summary statistics and a filtered table of site-species pairs that are not monitored, which can be used for further reporting or to update active site/species metadata.

In [11]:
def extract_month_number(month_str):
    # expects format 2025_jul returns 7 for 'jul
    month_map = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6,
                 'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12}
    m = re.match(r"\d{4}_(\w{3})", str(month_str).lower())
    if m:
        return month_map.get(m.group(1), None)
    return None

def analyse_affected_sites(output_filtered_value_file, output_notActive_site_species):
    """
    Scan for site-species combinations with 100% missing values for all months in a year.
    Prints summary and returns the grouped DataFrame.
    """
    # Load the CSV
    df = pd.read_csv(output_filtered_value_file, encoding='utf-8')
    print("CSV structure (columns):", df.columns.tolist())
    print("First 5 rows:\n", df.head())

    # Extract month number from 'month' column
    df['month_number'] = df['month'].apply(extract_month_number)

    # Remove rows where month conversion failed
    df = df[df['month_number'].notna()].copy()

    # Count files per site, species, year, and month
    df['count'] = 1
    summary = df.groupby(['siteCode', 'SpeciesCode', 'year', 'month']).size().reset_index(name='count')
    print("\nGrouped summary (site, species, year, month):")
    print(summary.head())

    # Pivot for wide view: months per year
    pivot = summary.pivot_table(index=['siteCode', 'SpeciesCode', 'year'], values='count', aggfunc='sum', fill_value=0)
    print("\nPivot table (site, species, year):")
    print(pivot.head())

    # Pivot to see years as columns (site/species as index)
    summary_year = df.groupby(['siteCode', 'SpeciesCode', 'year'])['count'].sum().reset_index()
    pivot_year = summary_year.pivot_table(index=['siteCode', 'SpeciesCode'], columns='year', values='count', fill_value=0)
    print("\nPivot table (site/species x year):")
    print(pivot_year.head())

    # Ensure columns are strings for year
    pivot_year.columns = pivot_year.columns.astype(str)
    required_years = ['2023', '2024', '2025']
    pivot_year = pivot_year[[col for col in required_years if col in pivot_year.columns]]

    # Filter for exact month counts: 2023 (12), 2024 (12), 2025 (11)
    filtered = pivot_year[
        (pivot_year.get('2023', 0) == 12) &
        (pivot_year.get('2024', 0) == 12) &
        (pivot_year.get('2025', 0) == 11)
    ]
    print("\nFiltered site/species with all months missing (2023:12, 2024:12, 2025:11):")
    print(filtered)

    # Save the filtered DataFrame to CSV (commented out for now)
    filtered.to_csv(output_notActive_site_species, index=True, encoding='utf-8')
    print(f"\nFiltered site/species combos saved to: {output_notActive_site_species}")

    return filtered

In [12]:
# use the function to analyse affected sites/species
filtered = analyse_affected_sites(input_all_missing_file, output_notActive_site_species)

CSV structure (columns): ['filename', 'path', 'siteCode', 'SpeciesCode', 'year', 'month', 'EmptyValuePercentage']
First 5 rows:
                               filename  \
0     BL0_CO_2023-04-01_2023-04-30.csv   
1     BT4_O3_2023-04-01_2023-04-30.csv   
2    BT4_SO2_2023-04-01_2023-04-30.csv   
3  BT5_PM2.5_2023-04-01_2023-04-30.csv   
4  BT6_PM2.5_2023-04-01_2023-04-30.csv   

                                                path siteCode SpeciesCode  \
0  /Users/burdzhuchaglayan/Desktop/data science p...      BL0          CO   
1  /Users/burdzhuchaglayan/Desktop/data science p...      BT4          O3   
2  /Users/burdzhuchaglayan/Desktop/data science p...      BT4         SO2   
3  /Users/burdzhuchaglayan/Desktop/data science p...      BT5       PM2.5   
4  /Users/burdzhuchaglayan/Desktop/data science p...      BT6       PM2.5   

   year     month  EmptyValuePercentage  
0  2023  2023_apr                100.00  
1  2023  2023_apr                 52.87  
2  2023  2023_apr            

### 3) Remove non-active site/species pairs from active metadata

This function updates the active site/species metadata by removing any pairs that are identified as non-active (i.e., those with 100% missing values for all months in a year).

- **Inputs:**
  - `actv_sites_species.csv`: The current list of active site/species pairs.
  - `notActive_site_species.csv`: The list of site/species pairs with no valid data (100% missing) for all months (2023: 12, 2024: 12, 2025: 11).

- **Process:**
  - Compares the active list to the non-active list.
  - Removes any rows from the active metadata where `(SiteName, SpeciesCode)` matches `(siteCode, SpeciesCode)` in the non-active list.

- **Output:**
  - Saves the updated active site/species list as `updated_actv_siteSpecies.csv` in the same directory.

This ensures that the active metadata only includes site/species pairs that are genuinely monitored, improving the accuracy of subsequent analyses and reporting.

In [11]:
def normalise_species_code(code):
    # Treat PM2.5 and PM25 as equivalent, and normalise case/whitespace
    code = str(code).strip().upper().replace('.', '')
    return code

def remove_nonactive_from_active():
    """
    Remove non-active site/species combinations from the active metadata list.
    - Reads actv_sites_species.csv (columns: SiteName, SpeciesCode)
    - Reads notActive_site_species.csv (columns: siteCode, SpeciesCode)
    - Removes any (SiteName, SpeciesCode) in the active list that matches (siteCode, SpeciesCode) in the non-active list.
    - Saves the updated list as updated_actv_siteSpecies.csv.
    - Prints a summary of removals for reporting and reproducibility.
    """
    # Load data
    df_active = pd.read_csv(actv_sites_species_path, encoding='utf-8')
    df_nonactive = pd.read_csv(output_notActive_site_species, encoding='utf-8')


    # Normalise species codes in both dataframes
    df_active['SpeciesCode_norm'] = df_active['SpeciesCode'].apply(normalise_species_code)
    df_nonactive['SpeciesCode_norm'] = df_nonactive['SpeciesCode'].apply(normalise_species_code)

    # Create set of non-active pairs (normalised)
    nonactive_set = set(zip(df_nonactive['siteCode'], df_nonactive['SpeciesCode_norm']))
    active_set = set(zip(df_active['SiteCode'], df_active['SpeciesCode_norm']))

    # Filter active DataFrame (normalised)
    filtered_active = df_active[
        ~df_active.apply(lambda row: (row['SiteCode'], row['SpeciesCode_norm']) in nonactive_set, axis=1)
    ].copy()

    # Calculate and print what was removed
    removed_count = len(df_active) - len(filtered_active)
    print("="*60)
    if removed_count > 0:
        print(f"{removed_count} non-active site/species pairs were removed from the active list.")
    else:
        print("No matching non-active site/species pairs found in the active list. No rows were removed.")
    print(f"Original rows: {len(df_active)}, Rows after removal: {len(filtered_active)}")
    print("="*60)

    # Save the filtered DataFrame
    filtered_active.drop(columns=['SpeciesCode_norm'], inplace=True)
    filtered_active.to_csv(updated_actv_site_species_path, index=False, encoding='utf-8')
    print(f"Filtered active data saved to: {updated_actv_site_species_path}")

    # Show what was removed
    removed_combinations = df_active[
        ~df_active.apply(lambda row: (row['SiteCode'], row['SpeciesCode_norm']) in set(
            zip(filtered_active['SiteCode'], filtered_active['SpeciesCode_norm'])
        ), axis=1)
    ][['SiteCode', 'SpeciesCode']].drop_duplicates()
    if not removed_combinations.empty:
        print("\nRemoved site/species combinations:")
        print(removed_combinations.to_string(index=False))

    # Report non-active pairs not present in the active list
    not_in_active = nonactive_set - active_set
    if not_in_active:
        print(f"\n{len(not_in_active)} site/species pairs in notActive_site_species.csv are not present in actv_sites_species.csv:")
        for site, species in sorted(not_in_active):
            print(f"  SiteCode: {site}, SpeciesCode: {species}")
    else:
        print("\nAll site/species pairs in notActive_site_species.csv are present in actv_sites_species.csv.")

    return filtered_active


In [12]:
# run the code
cleaned_df = remove_nonactive_from_active()

38 non-active site/species pairs were removed from the active list.
Original rows: 252, Rows after removal: 214
Filtered active data saved to: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/laqn/updated_actv_siteSpecies.csv


KeyError: 'SpeciesCode_norm'

 output of the code:

        ============================================================
        36 non-active site/species pairs were removed from the active list.
        Original rows: 252, Rows after removal: 216
        ============================================================

        Removed site/species combinations:
        SiteCode SpeciesCode
            BG1         NO2
            BG1         SO2
            BG2         NO2
            BG2        PM10
            BX2         NO2
            BX2        PM10
            BX2        PM25
            BQ7         NO2
            BQ7          O3
            BQ7        PM10
            BQ7        PM25
            BX1          CO
            BX1         NO2
            BX1          O3
            BX1        PM10
            BX1         SO2
            BQ9        PM10
            BQ9        PM25
            BT8         NO2
            BT8        PM10
            BT8        PM25
            BT4         NO2
            BT4          O3
            BT4        PM10
            BT4        PM25
            BT4         SO2
            BT6         NO2
            BT6        PM10
            BT6        PM25
            BT6         SO2
            BT5         NO2
            BT5        PM10
            BT5        PM25
            BY7          CO
            BY7         NO2
            BY7        PM10
            BY7        PM25
            BL0          CO
            BL0         NO2
            BL0          O3
            BL0        PM10
            BL0        PM25
            BL0         SO2
            CD1         NO2
            CD1        PM10
            CD1        PM25
            CR5         NO2
            CR8        PM25
            CR9         NO2
            CR9        PM10
            CR7         NO2
            EA6         NO2
            EA6        PM10
            EA8         NO2
            EA8        PM10
            EI8        PM10
            EI1         NO2
            EI1        PM10
            EN5         NO2
            EN5        PM10
            EN5        PM25
            EN1         NO2
            EN4         NO2
            EN4        PM10
            EN4         SO2
            EN7         NO2
            GN0         NO2
            GN0        PM10
            GN0        PM25
            GR7         NO2
            GR7        PM10
            GR4         NO2
            GR4          O3
            GR4        PM10
            GR4        PM25
            GR4         SO2
            GB6         NO2
            GB6          O3
            GB6        PM10
            GB0        PM10
            GB0        PM25
            GN4         NO2
            GN4        PM10
            GN6         NO2
            GN6        PM10
            GN6        PM25
            GN3         NO2
            GN3          O3
            GN3        PM10
            GN3        PM25
            GN5         NO2
            GN5        PM10
            TL4         NO2
            GR9         NO2
            GR9          O3
            GR9        PM10
            GR9        PM25
            GR8         NO2
            GR8        PM10
            GR8        PM25
            HK6         NO2
            HK6          O3
            HK6        PM10
            HK6        PM25
            HG4         NO2
            HG4          O3
            HG1         NO2
            HG1        PM10
            HG1        PM25
            HG1         SO2
            HV1         NO2
            HV1        PM10
            HV1        PM25
            HV3         NO2
            HV3        PM10
            HV3         SO2
            LH0          CO
            LH0         NO2
            LH0          O3
            LH0        PM10
            LH0        PM25
            HI0          CO
            HI0         NO2
            HI0          O3
            HI0        PM10
            HI0         SO2
            IS6         NO2
            IS6        PM10
            IS6        PM25
            IS2          CO
            IS2         NO2
            IS2        PM10
            IS2        PM25
            KC1          CO
            KC1         NO2
            KC1          O3
            KC1        PM10
            KC1        PM25
            KC1         SO2
            KF1        PM10
            KF1        PM25
            GT1         NO2
            GT1        PM10
            GT1        PM25
            LB5         NO2
            LB5        PM10
            LB5         SO2
            LB4         NO2
            LB4        PM10
            LB4        PM25
            LB4         SO2
            LB6         NO2
            LB6        PM10
            WAC         NO2
            WAC        PM10
            WAC        PM25
            HP1         NO2
            HP1          O3
            HP1        PM10
            HP1        PM25
            TD5        PM10
            TD5        PM25
            MR8         NO2
            MR8        PM10
            MR8        PM25
            ME2         NO2
            ME2        PM10
            ME2        PM25
            MEA         NO2
            MEA        PM25
            ME9         NO2
            MEB         NO2
            MEB        PM10
            MEB        PM25
            TL6         NO2
            TL6        PM25
            TL5         NO2
            CE3         NO2
            CE3        PM10
            CE3        PM25
            RI2         NO2
            RI2          O3
            RI2        PM10
            RI2        PM25
            RI1         NO2
            RI1        PM10
            RHI         NO2
            RHI        PM10
            RHI        PM25
            SK5         NO2
            SK5        PM10
            TH4         NO2
            TH4          O3
            TH4        PM10
            TH4        PM25
            CW3         NO2
            CW3        PM10
            CW3        PM25
            TH7         NO2
            TH7        PM25
            TH2          CO
            TH2         NO2
            TH2        PM25
            TH6         NO2
            TH6          O3
            TH6        PM10
            TH5         NO2
            TH5          O3
            TH5        PM10
            TH5        PM25
            TH5         SO2
            WAA         NO2
            WAA        PM10
            WAA        PM25
            WA9          CO
            WA9         NO2
            WA9        PM10
            WA9        PM25
            WA7         NO2
            WA7        PM10
            WA7        PM25
            WAB         NO2
            WAB        PM10
            WAB        PM25
            CE2         NO2
            CE2          O3
            CE2        PM10
            CE2        PM25
            WM5         NO2
            WM5        PM25
            WMD         NO2
            WMD        PM25
            WM0          CO
            WM0         NO2
            WM0          O3
            WM0        PM10
            WM0        PM25
            WM0         SO2
            MY1          CO
            MY1         NO2
            MY1          O3
            MY1        PM10
            MY1         SO2
            WM6         NO2
            WM6        PM10
            WM6        PM25
            WME         NO2
            WME          O3
            WME        PM25

        2 site/species pairs in notActive_site_species.csv are not present in actv_sites_species.csv:
        SiteCode: KF1, SpeciesCode: PM2.5
        SiteCode: MR8, SpeciesCode: PM2.5

    - Normalised species codes (e.g., PM2.5/PM25) for accurate matching. after that KF1 and MR8 PM2.5 also removed, from actv_site_species.csv.